In [1]:
from sklearn import tree
import pandas as pd
import os
import numpy as np

In [2]:
file = 'clt_temps.csv'


In [3]:
df = pd.read_csv(file, encoding="ISO-8859-1")

In [4]:
df.head()

,date_time,avg_temp,precip_water,snow,snow_ice_depth
0,1948-09-01,71.5,0.00,0.0,0.0
1,1948-09-02,69.0,0.00,0.0,0.0
2,1948-09-03,72.0,0.00,0.0,0.0
3,1948-09-04,69.0,0.29,0.0,0.0
4,1948-09-05,72.0,0.06,0.0,0.0


In [5]:
score_df = df.loc[:, ["date_time", "avg_temp", "precip_water", "snow", "snow_ice_depth"]]

In [6]:
# df.insert(7, 'score', value)
score_df.head()

,date_time,avg_temp,precip_water,snow,snow_ice_depth
0,1948-09-01,71.5,0.00,0.0,0.0
1,1948-09-02,69.0,0.00,0.0,0.0
2,1948-09-03,72.0,0.00,0.0,0.0
3,1948-09-04,69.0,0.29,0.0,0.0
4,1948-09-05,72.0,0.06,0.0,0.0


In [7]:
for index, row in score_df.head(5).iterrows():
    print(row['avg_temp'])

71.5
69.0
72.0
69.0
72.0


In [8]:
score_df["precip"] = score_df["snow"] + score_df["snow_ice_depth"] + score_df["precip_water"]
   

In [9]:
score_df.head(5)


,date_time,avg_temp,precip_water,snow,snow_ice_depth,precip
0,1948-09-01,71.5,0.00,0.0,0.0,0.00
1,1948-09-02,69.0,0.00,0.0,0.0,0.00
2,1948-09-03,72.0,0.00,0.0,0.0,0.00
3,1948-09-04,69.0,0.29,0.0,0.0,0.29
4,1948-09-05,72.0,0.06,0.0,0.0,0.06


In [10]:
conditions = [
    (score_df['precip'] == 0.00) & (score_df['avg_temp'] >= 70) & (score_df['avg_temp'] <= 80)]
choices = [2]
score_df['score'] = np.select(conditions, choices, default=1)

In [11]:
score_df.head(5)


,date_time,avg_temp,precip_water,snow,snow_ice_depth,precip,score
0,1948-09-01,71.5,0.00,0.0,0.0,0.00,2
1,1948-09-02,69.0,0.00,0.0,0.0,0.00,1
2,1948-09-03,72.0,0.00,0.0,0.0,0.00,2
3,1948-09-04,69.0,0.29,0.0,0.0,0.29,1
4,1948-09-05,72.0,0.06,0.0,0.0,0.06,1


In [12]:
score_df.head()

,date_time,avg_temp,precip_water,snow,snow_ice_depth,precip,score
0,1948-09-01,71.5,0.00,0.0,0.0,0.00,2
1,1948-09-02,69.0,0.00,0.0,0.0,0.00,1
2,1948-09-03,72.0,0.00,0.0,0.0,0.00,2
3,1948-09-04,69.0,0.29,0.0,0.0,0.29,1
4,1948-09-05,72.0,0.06,0.0,0.0,0.06,1


In [13]:
score_df.to_csv('scored.csv')

In [14]:
x = score_df["avg_temp"]
x.head()

0    71.5
1    69.0
2    72.0
3    69.0
4    72.0
Name: avg_temp, dtype: float64

In [15]:
y = score_df["score"]
y.head()

0    2
1    1
2    2
3    1
4    1
Name: score, dtype: int64

In [16]:
x= score_df.avg_temp.values.reshape(-1, 1)
# y= score_df.score.values.reshape(-1, 1)

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [18]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(x, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [19]:
# Fit to the training data and validate with the test data
model = SVC(kernel='linear')
model.fit(x_train, y_train)
predictions = model.predict(x_test)


In [20]:
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["good day", "bad day"]))

              precision    recall  f1-score   support

    good day       0.82      1.00      0.90      5345
     bad day       0.00      0.00      0.00      1155

   micro avg       0.82      0.82      0.82      6500
   macro avg       0.41      0.50      0.45      6500
weighted avg       0.68      0.82      0.74      6500



/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
